# SageMaker + Glue Interactive Sessions

In [3]:
%stop_session

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.37.2 and you have 0.37.0 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
There is no current session.


In [9]:
%additional_python_modules transformers,torch
%number_of_workers 10

Additional python modules to be included:
transformers
datasets
torch
Previous number of workers: 10
Setting new number of workers to: 10


In [1]:
import sys
print(sys.version)

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 10
Session ID: c78ff64c-28ab-409d-87b9-f53812c8bff4
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--additional-python-modules transformers,datasets,torch
Waiting for session c78ff64c-28ab-409d-87b9-f53812c8bff4 to get into ready status...
Session c78ff64c-28ab-409d-87b9-f53812c8bff4 has been created.
3.7.16 (default, Dec 15 2022, 23:24:54) 
[GCC 7.3.1 20180712 (Red Hat 7.3.1-15)]


In [6]:
import transformers, torch #, datasets

In [5]:
# from datasets import load_dataset
    
# datasets = load_dataset("amazon_us_reviews", "Digital_Video_Games_v1_00") #, data_files={"train": "Apparel_v1_00"})  #, "validation": path_to_validation.txt}

# datasets

Dataset amazon_us_reviews downloaded and prepared to /home/spark/.cache/huggingface/datasets/amazon_us_reviews/Digital_Video_Games_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date'],
        num_rows: 145431
    })
})
100%|##########| 1/1 [00:00<00:00, 192.75it/s]                                          


In [9]:
data = spark.read.parquet("s3://dsoaws/parquet/")

# The following command caches the DataFrame in memory. This improves performance since subsequent calls to the DataFrame can read from memory instead of re-reading the data from disk.
#df.cache()

                           PRE CFN_TEMPLATES_IS_USED_BY_DEEPLEARNING_AI_DO_NOT_DELETE/
                           PRE archive/
                           PRE autopilot/
                           PRE cfn_templates/
                           PRE cfn_templates_canvas/
                           PRE cfn_templates_hands_on/
                           PRE content-moderation/
                           PRE data-tfrecord-all/
                           PRE data/
                           PRE dataSharing/
                           PRE emr-studio/
                           PRE emr/
                           PRE feast/
                           PRE nyc-taxi-orig-cleaned-csv-with-header-all-years/
                           PRE nyc-taxi-orig-cleaned-csv-with-header-per-year/
                           PRE nyc-taxi-orig-cleaned-csv-without-header-all-years/
                           PRE nyc-taxi-orig-cleaned-csv-without-header-per-year/
                           PRE nyc-taxi-orig-cleaned-p

In [4]:
data.show(10)

+------------+-------------+---------+-------------------+-------------------+---------------+-------------+------------+------------------+------------+-----------+-----+-------+----------+------------+----+
|total_amount|      ride_id|vendor_id|          pickup_at|         dropoff_at|passenger_count|trip_distance|rate_code_id|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|year|
+------------+-------------+---------+-------------------+-------------------+---------------+-------------+------------+------------------+------------+-----------+-----+-------+----------+------------+----+
|        17.8|3049427049247|        1|2017-08-19 02:09:25|2017-08-19 02:23:43|              1|          4.2|           1|                 N|           1|       15.0|  0.5|    0.5|       1.5|         0.0|2017|
|       12.36|3049427049248|        2|2017-01-23 11:26:13|2017-01-23 11:39:54|              1|         1.31|           1|                 N|           1|        9.5

In [5]:
print("The dataset has %d rows." % data.count())

The dataset has 1054038765 rows.


In [5]:
print("The dataset has %d rows." % data.count())

The dataset has 1054038765 rows.


In [ ]:
df_avg_amount_by_passenger_count = data.select('passenger_count', 'total_amount') \
                                     .groupby('passenger_count') \
                                     .avg('total_amount') \
                                     .sort('passenger_count')
df_avg_amount_by_passenger_count.show()

In [18]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer

# Remove the target column from the input feature set.
featuresCols = data.columns
featuresCols.remove('total_amount')
featuresCols.remove('dropoff_at')
featuresCols.remove('pickup_at')
featuresCols.remove('store_and_fwd_flag')

In [23]:
# vectorAssembler combines all feature columns into a single feature vector column, "rawFeatures".
vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures", handleInvalid="skip")

# vectorIndexer identifies categorical features and indexes them, and creates a new column "features". 
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=100, handleInvalid="skip")

In [27]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
# featureIndexer =\
#     VectorIndexer(inputCol="rawFeatures", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Train a RandomForest model.
rf = RandomForestRegressor(labelCol="total_amount", featuresCol="features")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, rf])

In [28]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [29]:
# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [32]:
# Make predictions.
predictions = model.transform(testData)

In [33]:
# Select example rows to display.
predictions.select("prediction", "total_amount", "features").show(5)

+-------------------+------------+--------------------+
|         prediction|total_amount|            features|
+-------------------+------------+--------------------+
|-0.7248450083799632|      -127.8|[3.367254613393E1...|
| 11.817054733570682|      -120.3|[3.367255226508E1...|
|-0.7248450083799632|     -104.13|[3.367254605337E1...|
|-0.7248450083799632|      -100.8|[3.058017471584E1...|
|-0.7248450083799632|      -100.8|[3.367256683533E1...|
+-------------------+------------+--------------------+
only showing top 5 rows


In [ ]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="total_amount", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary only

## Preprocess data

The goal is to predict the `total_amount` (typically called the `fare`) of each ride.  To simplify the pre-processing, we may want to drop certain features like `pickup_at` and `dropoff_at` since taxi fares do not depend on the time of day, typically (unlike ride-share fares like Uber and Lyft).

We may also want to drop unused fields like `store_and_fwd_flag` which is an edge case where the taxi-meter was disconnected during the trip.  This should not impact the fare.

TODO:  Describe why we should drop the `rate_code_id` - or otherwise explain how it could be used.

Lastly, the `payment_type` are not useful for this predictive model as the fare should not depend on how the user is paying.

In [10]:
# # Data type string of column vendor_id is not supported.
# # Data type timestamp of column pickup_at is not supported.
# # Data type timestamp of column dropoff_at is not supported.
# # Data type string of column rate_code_id is not supported.
# # Data type string of column store_and_fwd_flag is not supported.
# # Data type string of column payment_type is not supported.

# df = df.drop("vendor_id").drop("pickup_at").drop("dropoff_at").drop("rate_code_id").drop("store_and_fwd_flag").drop("payment_type")
# df.show()

In [11]:
# df.printSchema()

#### Split data into training and test sets

Randomly split data into training and test sets. By doing this, you can train and tune the model using only the training subset, and then evaluate the model's performance on the test set to get a sense of how the model will perform on new data. 

In [12]:
# # Split the dataset randomly into 70% for training and 30% for testing. Passing a seed for deterministic behavior
# train, test = df.randomSplit([0.7, 0.3], seed = 0)
# print("There are %d training examples and %d test examples." % (train.count(), test.count()))

#### Visualize the data
You can plot the data to explore it visually. The following plot shows the number of bicycle rentals during each hour of the day.  As you might expect, rentals are low during the night, and peak at commute hours.  

To create plots, call `display()` on a DataFrame in Databricks and click the plot icon below the table.

To create the plot shown, run the command in the following cell. The results appear in a table. From the drop-down menu below the table, select "Line". Click **Plot Options...**. In the dialog, drag `hr` to the **Keys** field, and drag `cnt` to the **Values** field. Also in the **Keys** field, click the "x" next to `<id>` to remove it. In the **Aggregation** drop down, select "AVG". 

In [13]:
# train.select("passenger_count", "total_amount").show()

## Train the machine learning pipeline

Now that you have reviewed the data and prepared it as a DataFrame with numeric values, you're ready to train a model to predict future bike sharing rentals. 

Most MLlib algorithms require a single input column containing a vector of features and a single target column. The DataFrame currently has one column for each feature. MLlib provides functions to help you prepare the dataset in the required format.

MLlib pipelines combine multiple steps into a single workflow, making it easier to iterate as you develop the model. 

In this example, you create a pipeline using the following functions:
* `VectorAssembler`: Assembles the feature columns into a feature vector.
* `VectorIndexer`: Identifies columns that should be treated as categorical. This is done heuristically, identifying any column with a small number of distinct values as categorical.
* `SparkXGBRegressor`: Uses the [SparkXGBRegressor](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.spark.SparkXGBRegressor) estimator to learn how to predict the fare from the feature vectors.
* `CrossValidator`: The XGBoost regression algorithm has several hyperparameters. This notebook illustrates how to use [hyperparameter tuning in Spark](https://spark.apache.org/docs/latest/ml-tuning.html). This capability automatically tests a grid of hyperparameters and chooses the best resulting model.

For more information:  
[VectorAssembler](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler)  
[VectorIndexer](https://spark.apache.org/docs/latest/ml-features.html#vectorindexer)  

The first step is to create the VectorAssembler and VectorIndexer steps. 

In [14]:
# from pyspark.ml.feature import VectorAssembler, VectorIndexer

# # Remove the target column from the input feature set.
# featuresCols = df.columns
# featuresCols.remove('total_amount')

# # vectorAssembler combines all feature columns into a single feature vector column, "rawFeatures".
# vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures", handleInvalid="skip")

# # vectorIndexer identifies categorical features and indexes them, and creates a new column "features". 
# vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=100, handleInvalid="skip")

Next, define the model. To use distributed training, set `num_workers` to the number of spark tasks you want to concurrently run during training xgboost model.

In [16]:
# !pip install xgboost==1.6.2

In [ ]:
# import xgboost
# print(xgboost.__version__)

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 10
Session ID: 98460f0e-a17a-489e-8d14-86a479846284
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--additional-python-modules #xgboost==1.7.1
Waiting for session 98460f0e-a17a-489e-8d14-86a479846284 to get into ready status...


In [18]:
# from xgboost.spark import SparkXGBRegressor

# # The next step is to define the model training stage of the pipeline. 
# # The following command defines a XgboostRegressor model that takes an input column "features" by default and learns to predict the labels in the "cnt" column.
# # Set `num_workers` to the number of spark tasks you want to concurrently run during training xgboost model.
# xgb_regressor = SparkXGBRegressor(label_col="total_amount", missing=0.0)

The third step is to wrap the model you just defined in a `CrossValidator` stage. `CrossValidator` calls the XgboostRegressor estimator with different hyperparameter settings. It trains multiple models and selects the best one, based on minimizing a specified metric. In this example, the metric is [root mean squared error (RMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation).


In [19]:
# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
# from pyspark.ml.evaluation import RegressionEvaluator

# # Define a grid of hyperparameters to test:
# #  - maxDepth: maximum depth of each decision tree 
# #  - maxIter: iterations, or the total number of trees 
# paramGrid = ParamGridBuilder()\
#   .addGrid(xgb_regressor.max_depth, [2, 5])\
#   .addGrid(xgb_regressor.n_estimators, [10, 100])\
#   .build()

# # Define an evaluation metric.  The CrossValidator compares the true labels with predicted values for each combination of parameters, and calculates this value to determine the best model.
# evaluator = RegressionEvaluator(metricName="rmse",
#                                 labelCol=xgb_regressor.getLabelCol(),
#                                 predictionCol=xgb_regressor.getPredictionCol())

# # Declare the CrossValidator, which performs the model tuning.
# cv = CrossValidator(estimator=xgb_regressor, evaluator=evaluator, estimatorParamMaps=paramGrid)

Create the pipeline.

In [20]:
# from pyspark.ml import Pipeline
# pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])

In [20]:
# from pyspark.ml import Pipeline
# pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])

Train the pipeline.

Now that you have set up the workflow, you can train the pipeline with a single call.  
When you call `fit()`, the pipeline runs feature processing, model tuning, and training and returns a fitted pipeline with the best model it found.
This step takes several minutes.

In [21]:
# pipelineModel = pipeline.fit(train)

## Make predictions and evaluate results

The final step is to use the fitted model to make predictions on the test dataset and evaluate the model's performance. The model's performance on the test dataset provides an approximation of how it is likely to perform on new data.

Computing evaluation metrics is important for understanding the quality of predictions, as well as for comparing models and tuning parameters.

In [22]:
# predictions = pipelineModel.transform(test)

In [22]:
# predictions = pipelineModel.transform(test)

In [23]:
# predictions.select("total_amount", "prediction", *featuresCols).show()

The `transform()` method of the pipeline model applies the full pipeline to the input dataset. The pipeline applies the feature processing steps to the dataset and then uses the fitted XGBoost Regressor model to make predictions. The pipeline returns a DataFrame with a new column `predictions`.

A common way to evaluate the performance of a regression model is the calculate the [root mean squared error (RMSE)](https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#regression-model-evaluation). The value is not very informative on its own, but you can use it to compare different models. `CrossValidator` determines the best model by selecting the one that minimizes RMSE. 

In [24]:
# rmse = evaluator.evaluate(predictions)
# print("RMSE on our test set: %g" % rmse)

In [24]:
# rmse = evaluator.evaluate(predictions)
# print("RMSE on our test set: %g" % rmse)

You can also plot the results, as you did the original dataset. In this case, the hourly count of rentals shows a similar shape.

In [25]:
# predictions.select("passenger_count", "prediction").show()

## Save and reload the model


In [8]:
#%%sh

#rm -rf /dbfs/tmp/xgboost/pipeline_001
#rm -rf /dbfs/tmp/xgboost/pipelineModel_001

UsageError: %%sh is a cell magic, but the cell body is empty.
session_id=efa0d3dc-d13c-48ee-8f9c-7ee94a131c8f has reached TIMEOUT status. 
Please re-run the same cell to restart the session. You may also need to re-run previous cells if trying to use pre-defined variables.


In [28]:
# # Save the pipeline that created the model
# pipeline.save('/tmp/xgboost/pipeline_001')

# # Save the model itself
# pipelineModel.save('/tmp/xgboost/pipelineModel_001')

In [29]:
# # Load the pipeline
# loaded_pipeline = Pipeline.load('/tmp/xgboost/pipeline_001')

In [30]:
# # Load and use the model
# from pyspark.ml import PipelineModel

# loaded_pipelineModel = PipelineModel.load('/tmp/xgboost/pipelineModel_001')

# # To represent new data, use the first 3 rows of the test dataset
# new_data = test.limit(3)

# # Make predictions with the loaded model
# new_preds = loaded_pipelineModel.transform(new_data)
# display(new_preds.select("total_amount", "prediction", *featuresCols))